## Documentation and Interesting Results from Code Speed Up

### Setting up the examples to run through the forward modeller

In [1]:
%matplotlib inline
import sys
myBase="D:\\WorkForOthers\\USGS\\MCMC\\Python"
if myBase not in sys.path:
    sys.path.append(myBase)
import setupPackage
from FdemData import FdemData
from Stopwatch import Stopwatch
from StatArray import StatArray
#from System import EmSystem
from Model1D import Model1D
from scipy.constants import mu_0 as mu0
from scipy.constants import epsilon_0 as eps0
import numpy as np
import cmath as cm
import Error as Err
import time as time
import matplotlib.pyplot as plt
N=100000

In [2]:
# The data file name
dataFile='../Data/FdemData.csv'
# The EM system file name
systemFile="hemSystem.txt"
D=FdemData()
D.read(dataFile,systemFile)
P=D.getDataPoint(0)
P.z[:]=35.2

# Create a model with the # of layers as input
con=[0.000392628452446,0.000325149730765,   0.000382893410534,   0.000406282291944,   0.000000000977662,   0.000036199582044,   0.000000000005504,   0.017405471215531,   0.012247158625986,   0.071007642063211,   0.054715322480840,   0.016057998091585,   0.001402874523406,   0.000000948808653,   0.000000001403383,   0.000602024538505,   0.082170028582427,   0.000000889045405,   0.002039497239736,   0.000021128714358,   0.000000000366016,   0.000579847880082,   0.000000000063988,   0.029142208651212,   0.213002720466422,   0.027305520944434,   0.184039623884463,   0.000000000167473,   0.740817138328565,   0.000000000124133]
thk=[1.506304835312014,   2.545858145434390,   5.025650447739466,   4.951042896904797,   4.148059319040655,   5.478954167713898,   3.117559623909347,   5.473159265644906,   4.068959105263104,  13.206568558595997,   7.482754829287153,   8.728005290161825,   5.198880117483057,   5.261538801714735,   3.272546111675439,   4.569255431822512,   2.517115172736155,   4.229317958524064,  10.377421445020047,   7.193529306967264,   3.841476894831672,   6.382407545807752,   6.662544568868938,   3.826471571459606,   2.973877306345486,   4.819340786030267,   5.161285840207938,   3.893569754464238,   3.498839469107594,                   0]
Mod=Model1D()
Mod.maketest(29)
Mod.par[:]=con
Mod.thk[:]=thk

S=P.sys
mod=Mod

In [3]:
# Run the forward modeller to get a base line
clk=Stopwatch('New Code') 
clk.start()
P.forward(Mod)
print(clk.lap())
print(P.p[:])
J=P.sensitivity(Mod)
print(clk.lap())
clk.stop()
  
print(clk)

tmp=P.p-[22.4172462541,90.5961362227,54.757048434,228.628573101,321.578579083,414.65043221,45.1892248798,103.995583076,44.8479324477,114.202074154,114.75538919,162.320269454]
print('Answers are the same? '+str(np.linalg.norm(tmp) < 1e-6))

0.04347085952758789
[  22.41724624   90.59613622   54.75704843  228.6285731   321.57857908
  414.65043221   45.18922488  103.99558308   44.84793245  114.20207415
  114.75538919  162.32026945]
0.08953714370727539
Elapsed Time: 00:00:00.133 (h:m:s.ms)
Answers are the same? True


# fdem1dfwd()

In [4]:
z0=-P.z[0]
if (z0 > mod.top):
    Err.Emsg("Sensor altitude must be above the top of the model")

# Create the indices of the coil orientations for the frequencies.
xx,xy,xz,yx,yy,yz,zx,zy,zz = S.getComponentID()
useJ0=False
useJ1=False
if (len(xx),len(xy),len(yx),len(yy),len(zz) != 0):
  useJ0=True
if (len(xx),len(xy),len(xz),len(yx),len(yy),len(yz),len(zx),len(zy) != 0):
  useJ1=True

rTEj0Flg1 = []
u0j0Flg1 = []
drTEdlogconj0Flg1 = []
rTEj1Flg1 = []
u0j1Flg1 = []
drTEdlogconj1Flg1 = []  

### Speed up regarding the use of expanding lists or pre-allocated numpy arrays
#### Even though the iterations are many, the time savings may be dramatic for large algorithms

In [5]:
# Original empty list and expansion
t1=time.time()
for i in range(N):
    tMom=[];rMom=[]
    tMom = [S.T[i].moment for i in range(S.nFreq)]
    rMom = [S.R[i].moment for i in range(S.nFreq)]
    scl1 = np.asarray(tMom) * np.asarray(rMom)
ta=time.time()-t1
print(ta)
t1=time.time()
# Preallocation using numpy and maintaining a numpy array
for i in range(N):
    tMom=np.zeros(S.nFreq);rMom=np.zeros(S.nFreq)
    for i in range(S.nFreq):
      tMom[i] = S.T[i].moment
      rMom[i] = S.R[i].moment
    scl = tMom*rMom
tb=time.time()-t1
print(tb)
print('Answers are the same? '+str(np.linalg.norm(scl-scl1)==0.0))
print('Speed up: '+str(ta/tb))

1.1632590293884277
1.2194480895996094
Answers are the same? True
Speed up: 0.9539225484951716


## calcTEsens

In [6]:
S
J=S.H.J0
M=Mod
computeSens=False

In [7]:
# Get the number of layers in the model
nLayers=M.nCells[0]
nL1=nLayers+1

lamSq=np.zeros(J.lam.shape)
lamSq[:]=J.lam**2.0

# Compute the angular frequencies from the system frequencies 
omega=np.zeros(S.nFreq)
omega+=2.0*np.pi*S.freq[:]

### Another example of list expansion, this time numpy.insert.
#### Instead we can simply initialize an array with one extra length and have explicit assignment

In [8]:
# Create temporary variables and insert parameters at the beginning
# Original using np.insert
t1=time.time()
for i in range(N):
  con  = np.insert(M.par, 0, 0.0)
  #chie = np.insert(M.chie, 0, 0.0)
  chim = np.insert(M.chim, 0, 0.0)
  thk = np.insert(M.thk, 0, np.NAN)
ta=time.time()-t1
print(ta)
# Reworked with initialization with +1 and assignment
t1=time.time()
for i in range(N):
    con=np.zeros(nLayers+1);con[0]=0.0;con[1:]=M.par[:]
    chim=np.zeros(nLayers+1);chim[0]=0.0;chim[1:]=M.chim[:]
    thk=np.zeros(nLayers+1);thk[0]=np.NAN;thk[1:]=M.thk[:]
tb=time.time()-t1
print(tb)
print('Speed up: '+str(ta/tb))

5.4534289836883545
1.465066909790039
Speed up: 3.7223071159732175


### Building the responses per layer

In [9]:
# Build an array of responses per layer
t1=time.time()

tmp=StatArray([nL1,S.nFreq],"tmp","N/A",np.complex128)
for i in range(nL1):
    tmp[i,:]=np.dot(np.dot(1j,omega),1.0)
        
# Compute the Admitivity, yn=j*omega*eps+sigma
ynO=tmp*eps0
for i in range(nL1):
    ynO[i,:]=ynO[i,:]+con[i]
        
ynPO=StatArray([nL1,S.nFreq,J.nC],"ynP","N/A",np.complex128)
for i in range(J.nC):
    ynPO[:,:,i]=ynO[:,:]
    
# Compute the impedivity, zn=j*omega*mu
znO=tmp*mu0
znPO=StatArray([nL1,S.nFreq,J.nC],"znP","N/A",np.complex128)
for i in range(J.nC):
    znPO[:,:,i]=znO[:,:]
    
# Compute the wavenumber, un=sqrt(kx^2 + ky^2 - kn^2) = sqrt(lam^2 - kn^2)
# Where, kn = sqrt(-zn*yn) = sqrt(omega^2*mu_n*eps_n - j*omega*mu_n*sigma_n)
# Hence, un=sqrt(Lam^2+zn*yn)
unO=StatArray([nL1,S.nFreq,J.nC],"un","N/A",np.complex128)
for i in range(nL1):
    unO[i,:,:]=np.sqrt(lamSq+(znPO[i,:,:]*ynPO[i,:,:]))
        
# Set layer admittances, Yn
# eqn 4.27
YnO =  unO / znPO
ta=time.time()-t1
print(ta)

0.006012916564941406


#### Original python script to build ynP

In [10]:
# Try using cmath instead of casting to complex!!!
N=1
ynO = np.asarray([np.dot(np.dot(1j,omega),eps0)] * len(1+chim))
ynO = [ynO[i]+con[i] for i in range(len(ynO))]
t1=time.time()
for i in range(N):
  ynPO = np.asarray([[[itm2] * J.nC for itm2 in itm] for itm in ynO])
print('time to build ynP: '+str(time.time()-t1))


# Try using cmath instead of casting to complex!!!   
znO = [np.dot(np.dot(1j,omega),mu0)] * len((1+chim))
znO = [znO[i] for i in range(len(znO))]
t1=time.time()
for i in range(N):      
  znPO = np.asarray([[[itm2] * J.nC for itm2 in itm] for itm in znO])
print('time to build znP: '+str(time.time()-t1))

time to build ynP: 0.0029799938201904297
time to build znP: 0.005390167236328125


#### Yn section

In [11]:
# Original
N=1000
t1=time.time()
for i in range(N):
  tmp=StatArray([nL1,S.nFreq],"tmp","N/A",np.complex128)
  for i in range(nL1):
      tmp[i,:]=np.dot(np.dot(1j,omega),1.0)
         
  # Compute the Admitivity, yn=j*omega*eps+sigma
  ynO=tmp*eps0
  for i in range(nL1):
      ynO[i,:]=ynO[i,:]+con[i]
t0=time.time()-t1
print('Old method time: '+str(t0))
# 
t1=time.time()
for i in range(N):
  tmp=np.dot(1j,omega)
  ynA=np.zeros([nL1,S.nFreq],np.complex128,order='F')
  for i in range(nL1):
      ynA[i,:]=tmp*eps0+con[i]
ta=time.time()-t1
print('First try time: '+str(ta))

t1=time.time()
for i in range(N):
  tmp=np.dot(1j,omega)
  yn=np.repeat(tmp[:,np.newaxis]*eps0, nL1, 1)+con
tb=time.time()-t1
print('Final attempt time: '+str(tb))
print('Speed Up: '+str(t0/tb))
print('Previous Single iteration time: '+str(t0/N))
print('Single iteration time: '+str(tb/N))
print('Answers are the same? '+str(np.linalg.norm(ynO-yn.T)==0.0))

Old method time: 0.6299281120300293
First try time: 0.1389319896697998
Final attempt time: 0.01607489585876465
Speed Up: 39.18707266066476
Previous Single iteration time: 0.0006299281120300293
Single iteration time: 1.607489585876465e-05
Answers are the same? True


#### YnP section

In [12]:
t1=time.time()
for i in range(N):
  ynPO=np.zeros([nLayers+1,S.nFreq,J.nC],np.complex128)
  for i in range(J.nC):
    ynPO[:,:,i]=ynO[:,:]
t0=time.time()-t1
print('Time for orginal: '+str(t0))

Time for orginal: 0.4810049533843994


In [13]:
t1=time.time()
for i in range(N):
  ynPA=np.zeros([nLayers+1,S.nFreq,J.nC],np.complex128,order='F')
  for i in range(J.nC):
    ynPA[:,:,i]=yn.T[:,:]
ta=time.time()-t1
print('Time for first attempt: '+str(ta))
print('Answers are the same? '+str(np.linalg.norm(ynPA-ynPO)==0.0))

t1=time.time()
for i in range(N):
  ynP=np.repeat(yn[:,np.newaxis,:],J.nC,axis=1)
tb=time.time()-t1
print('Time for final attempt: '+str(tb))
print('Speed Up: '+str(t0/tb))
print('Single iteration time: '+str(tb/N))
dum=ynP.swapaxes(0,1)
dum=dum.swapaxes(0,2)
print('Answers are the same? '+str(np.linalg.norm(dum-ynPO)==0.0))

Time for first attempt: 0.2503218650817871
Answers are the same? True
Time for final attempt: 0.03087902069091797
Speed Up: 15.577079279780104
Single iteration time: 3.087902069091797e-05
Answers are the same? True


#### Zn Section

In [14]:
tmp=StatArray([nLayers+1,S.nFreq],"tmp","N/A",np.complex128)
for i in range(nLayers+1):
  tmp[i,:]=np.dot(np.dot(1j,omega),1.0)
# Compute the impedivity, zn=j*omega*mu
znO=tmp*mu0
znPO=StatArray([nLayers+1,S.nFreq,J.nC],"znP","N/A",np.complex128)
for i in range(J.nC):
  znPO[:,:,i]=znO[:,:]

In [15]:
t1=time.time()
for i in range(N):
  tmp=np.dot(1j,omega)
  zn=np.repeat(tmp[:,np.newaxis]*mu0, nL1, 1)
tb=time.time()-t1
print('Time: '+str(tb))
print('Answers are the same? '+str(np.linalg.norm(znO-zn.T)==0.0))

Time: 0.012647151947021484
Answers are the same? True


In [16]:
t1=time.time()
for i in range(N):
  znP=np.repeat(zn[:,np.newaxis,:],J.nC,axis=1)
  #znPB=znPB.swapaxes(0,2)
tb=time.time()-t1
print('Time: '+str(tb))
dum=znP.swapaxes(0,1)
dum=dum.swapaxes(0,2)
print('Answers are the same? '+str(np.linalg.norm(znPO-dum)==0.0))

Time: 0.038178205490112305
Answers are the same? True


#### Un Section

In [17]:
# Old section of code
t1=time.time()
for i in range(N):
  unO=StatArray([nLayers+1,S.nFreq,J.nC],"un","N/A",np.complex128)
  for i in range(nLayers+1):
    unO[i,:,:]=np.sqrt(lamSq+(znPO[i,:,:]*ynPO[i,:,:]))
tb=time.time()-t1
print('Time: '+str(tb))


Time: 1.2904410362243652


In [18]:
# New section
from math import sqrt
t1=time.time()
for i in range(N):
  zy=np.multiply(zn,yn)
  un=np.sqrt(np.repeat(zy[:,np.newaxis,:],J.nC,axis=1)+lamSq[:,:,np.newaxis])
ta=time.time()-t1
print('Time: '+str(ta))
dum=un.swapaxes(0,1)
dum=dum.swapaxes(0,2)
print('Speed up: '+str(tb/ta))
print('Answers are the same? '+str(np.linalg.norm(unO-dum)==0.0))

Time: 0.5643892288208008
Speed up: 2.2864381003877967
Answers are the same? True


In [19]:
Yn=un/znP

In [20]:
# Old section of code
t1=time.time()
for i in range(N):
  YO = YnO[1:].tolist()      
  # Insert the response from the airlayer
  YO.insert(0, np.ones((S.nFreq, J.nC)) * np.float64('NaN'))
ta=time.time()-t1
print('Time using toList(): '+str(ta))

Time using toList(): 0.6135752201080322


In [21]:
# New section
t1=time.time()
for i in range(N):
  Y=np.zeros([S.nFreq,J.nC,nL1],dtype=np.complex128)
  Y[:,:,nLayers]=Yn[:,:,nLayers]
  Y[:,:,0]=np.NAN
tb=time.time()-t1
print('Time without lists: '+str(tb))
print('Speed up: '+str(ta/tb))
print('Single iteration time: '+str(tb/N))

Time without lists: 0.03302907943725586
Speed up: 18.576818687109302
Single iteration time: 3.302907943725586e-05


#### Forward Modelling Options

In [22]:
def M0_0(Yn,Y,un):
  u0=un[:,:,0]
  #print(un.shape)
  #print(u0.shape)
  rTE=(Yn[:,:,0]-Y[:,:,1])/(Yn[:,:,0]+Y[:,:,1])
  return (rTE,u0)

In [23]:
def M1_0(Yn,Y,un,thk):
    nL=thk.size-1
    for i in reversed(range(1,nL)):
      tanuh=np.tanh(un[:,:,i]*thk[i])
      #tanuh=cm.tanh(unA[:,:,i]*thk[i])
      num=Y[:,:,i+1]+(Yn[:,:,i]*tanuh)
      den=Yn[:,:,i]+(Y[:,:,i+1]*tanuh)
      Y[:,:,i]=Yn[:,:,i]*(num/den)
    return M0_0(Yn,Y,un)

In [24]:
N=1000
t1=time.time()
for i in range(N):
  Y[:,:,nLayers]=Yn[:,:,nLayers]
  Y[:,:,0]=np.NAN
  rTE,u0 = M1_0(Yn,Y,un,thk)
tb=time.time()-t1
print('Time for new code: '+str(tb))

Time for new code: 2.7306759357452393


#### Sensitivity Options

In [25]:
N=10000
# Old code
t1=time.time()
for i in range(N):
  dYdlogcon = con[-1] / (np.dot(2.0,un[1:]))
  dYdlogcon = dYdlogcon.tolist()
  dYdlogcon.insert(0, (np.ones((S.nFreq, J.nC)) * float('NaN')).tolist())
ta=time.time()-t1
print('Time for old code: '+str(ta))

Time for old code: 7.563003063201904


In [26]:
# New code
t1=time.time()
for i in range(N):
  dydp=np.zeros([S.nFreq,J.nC,nL1],dtype=np.complex128)
  dydp[:,:,-1]=con[-1]/(2.0*un[:,:,-1])
  dydp[:,:,0]=np.NAN
tb=time.time()-t1
print('Time for new code: '+str(tb))
print('Speed up: '+str(ta/tb))

Time for new code: 0.3552231788635254
Speed up: 21.290849002022934


In [27]:
def M0_1(Yn,Y,un,dydp):
    rTE,u0=M0_0(Yn,Y,un)
    sens=-2.0*Yn[:,:,0]*dydp[:,:,1]/(Yn[:,:,0] + Y[:,:,1])**2.0
    return (rTE,u0,sens)

In [28]:
N=10000
t1=time.time()
for i in range(N):
  Y[:,:,nLayers]=Yn[:,:,nLayers]
  Y[:,:,0]=np.NAN
  rTE,u0,sens = M0_1(Yn,Y,un,dydp)
tb=time.time()-t1
print('Time for new code: '+str(tb))

Time for new code: 0.5651381015777588


In [29]:
def M1_1(Yn,Y,un,tmp,thk,con,dydp):
    dum=np.repeat(tmp[:,np.newaxis],np.size(Yn,1),1)*mu0
    dydy = np.ones(dydp.shape,dtype=np.complex128)
    nL=thk.size-1
    for i in reversed(range(1,nL)):
      i1=i+1
      tanuh=np.tanh(un[:,:,i]*thk[i])
      num=Y[:,:,i1]+(Yn[:,:,i]*tanuh)
      den=Yn[:,:,i]+(Y[:,:,i1]*tanuh)
      Y[:,:,i]=Yn[:,:,i]*(num/den)
      # Temporary variables
      dum1=dum*thk[i]   ; a0=Y[:,:,i1]**2.0
      b0=Yn[:,:,i]**2.0 ; c0=tanuh**2.0
      a1=Yn[:,:,i]*a0   ; a2=Yn[:,:,i]**3.0
      a3=a1-a2 ; t1=(con[i]/(2.0*un[:,:,i]*(den**2.0)))
      t6=dum1*a3
      t2=(2.0*Yn[:,:,i]*Y[:,:,i1]*c0) ; t3=t6*c0
      t4=((a0-b0)*tanuh)+2.0*b0 ; t5=(-t6)
      # dY(i)/d(ln(con(i)))
      dydp[:,:,i-1]=t1 *(t2 + t3 + t4 + t5)
      # dY(i)/dY(i+1)
      num1=b0*(1.0-c0)
      den1=den**-2.0
      dydy[:,:,i]=num1*den1
    u0=un[:,:,0]
    dum=(Yn[:,:,0]+Y[:,:,1])**-1.0
    rTE=(Yn[:,:,0]-Y[:,:,1])*dum
    dum1=Yn[:,:,0]
    num=-2.0*np.repeat(dum1[:,:,np.newaxis],nLayers,2)
    dum1=dum**2.0
    den=np.repeat(dum1[:,:,np.newaxis],nLayers,2)
    sens=num*dydp*np.cumprod(dydy,2)*den
    return (rTE,u0,sens)

In [30]:
N=100
t1=time.time()
for i in range(N):
  # Initialize
  Y[:,:,nLayers]=Yn[:,:,nLayers]
  Y[:,:,0]=np.NAN
  dydp=np.zeros([S.nFreq,J.nC,nLayers],dtype=np.complex128)
  dydp[:,:,nLayers-1]=con[-1]/(2.0*un[:,:,nLayers])    
  rTE,u0,sens=M1_1(Yn,Y,un,tmp,thk,con,dydp)
tb=time.time()-t1
print('Time: '+str(tb))
print('This is half time of matlab')

Time: 0.5729730129241943
This is half time of matlab


In [31]:
def calcrTEsens(S, J, M, computeSens):
    """ Calculate the reflection coefficient according to Ward and Hohmann, EM theory for geophysical applications
    B. Minsley, June 2010
    S:  : EmSystem Class
    J:  : Bessel Function Filter Parameters
    M:  : 1D Layered Model
    computeSens: : True or False
    """
    # Get the number of layers in the model
    nLayers=M.nCells[0]
    nL1=nLayers+1

    lamSq=J.lam**2.0

    # Compute the angular frequencies from the system frequencies 
    omega=np.zeros(S.nFreq)
    omega+=2.0*np.pi*S.freq[:]

    # Create temporary variables and insert parameters at the beginning
    con=np.zeros(nLayers+1);con[0]=0.0;con[1:]=M.par[:]
    chim=np.zeros(nLayers+1);chim[0]=0.0;chim[1:]=M.chim[:]
    thk=np.zeros(nLayers+1);thk[0]=np.NAN;thk[1:]=M.thk[:]
       
    # Build an array of responses per layer
    tmp=np.dot(1j,omega)
    # Compute the Admitivity, yn=j*omega*eps+sigma
    yn=np.repeat(tmp[:,np.newaxis]*eps0, nL1, 1)+con
#    ynP=np.repeat(yn[:,:,np.newaxis],J.nC,axis=2)
    zn=np.repeat(tmp[:,np.newaxis]*mu0, nL1, 1)
    znP=np.repeat(zn[:,np.newaxis,:],J.nC,axis=1)
    zy=np.multiply(zn,yn)
    un=np.sqrt(np.repeat(zy[:,np.newaxis,:],J.nC,axis=1)+lamSq[:,:,np.newaxis])
    Yn=un/znP
    
    Y=np.zeros([S.nFreq,J.nC,nL1],dtype=np.complex128)
    Y[:,:,nLayers]=Yn[:,:,nLayers]
    Y[:,:,0]=np.NAN
        
    if computeSens == 0:
      if nLayers == 1:
        return M0_0(Yn,Y,un) 
      return M1_0(Yn,Y,un,thk) 
    else:
      dydp=np.zeros([S.nFreq,J.nC,nLayers],dtype=np.complex128)
      dydp[:,:,nLayers-1]=con[-1]/(2.0*un[:,:,nLayers])    
      if nLayers == 1:
        return M0_1(Yn,Y,un,dydp)  
      return M1_1(Yn,Y,un,tmp,thk,con,dydp)

#### Sanity Check

In [32]:
# calculate reflection coefficient
if useJ0:
  # *** calcrTEsens.py only returns two variables for flg == 0 *** 
  varargin = calcrTEsens(S, S.H.J0, mod, computeSens)
  rTEj0 = varargin[0]
  u0j0 = varargin[1]
            
if useJ1:
  # *** calcrTEsens.py only returns two variables for flg == 0 *** 
  varargin = calcrTEsens(S, S.H.J1, mod, computeSens)
  rTEj1 = varargin[0]
  u0j1 = varargin[1]

In [33]:
# calculate reflection coefficient
if useJ0:
  # *** calcrTEsens.py only returns two variables for flg == 1 *** 
  varargin = calcrTEsens(S, S.H.J0, mod, True)
  rTEj0Flg1=(varargin[0])
  u0j0Flg1=(varargin[1])
  drTEdlogconj0Flg1=(varargin[2])
if useJ1:
  # *** calcrTEsens.py only returns two variables for flg == 1 *** 
  varargin = calcrTEsens(S, S.H.J1,mod, True)
  rTEj1Flg1=(varargin[0])
  u0j1Flg1=(varargin[1])
  drTEdlogconj1Flg1=(varargin[2])

#### Hzz Section

In [34]:
def calcHzz(i,S,z,rTEin,u0in,J):
  # Parse out the necessary components
  nComps=len(i)
  height=np.zeros(nComps) ; rHeight=np.zeros(nComps)
  tMom=np.zeros(nComps)
  for j in range(nComps):
    height[j]=-z+S.T[i[j]].off
    rHeight[j]=z+S.R[i[j]].off
    tMom[j]=S.T[i[j]].moment
  H=np.repeat(height[:,np.newaxis],J.nC,1)
  Z=np.repeat(rHeight[:,np.newaxis],J.nC,1)
  dist=S.dist[i]
  rTE=rTEin[i,:]
  u0=u0in[i,:]
  lam=J.lam[i,:]
  # Temporary variables
  a0=np.exp(-u0*(Z+H))
  a1=lam**3.0/u0
  a2=(tMom/(4.0*np.pi))/dist
  # Equation 4.46K(lam)
  K=(a0+rTE*np.exp(u0*(Z-H)))*a1
  Hzz=a2*(np.dot(K,J.W[:]))
  # Free-Space response
  K0=a0*a1
  Hzz0=a2*(np.dot(K0,J.W[:]))
  return Hzz,Hzz0

In [35]:
Hzz,Hzz0=calcHzz(zz,S,z0,rTEj0,u0j0,S.H.J0)

In [36]:
def calcHxx(i,S,z,rTEin0,rTEin1,u0in0,u0in1,J0,J1):
  # Parse out the necessary components
  nComps=len(i)
  height=np.zeros(nComps) ; rHeight=np.zeros(nComps)
  tMom=np.zeros(nComps)
  rx=np.zeros(nComps)
  for j in range(nComps):
    height[j]=-z+S.T[i[j]].off
    rHeight[j]=z+S.R[i[j]].off
    tMom[j]=S.T[i[j]].moment
    rx[j]=S.R[i[j]].tx    
  dist=S.dist[i]
  H0=np.repeat(height[:,np.newaxis],J0.nC,1)
  Z0=np.repeat(rHeight[:,np.newaxis],J0.nC,1)   
  lam0=J0.lam[i,:]
  rTE0=rTEin0[i,:]
  u0=u0in0[i,:]   
  H1=np.repeat(height[:,np.newaxis],J1.nC,1)
  Z1=np.repeat(rHeight[:,np.newaxis],J1.nC,1)     
  lam1=J1.lam[i,:]
  rTE1=rTEin1[i,:]
  u1=u0in1[i,:]
  # Temporary variables
  a0=np.exp(-lam0*(Z0+H0))
  a1=lam0**2.0
  b0=np.exp(-lam1*(Z1+H1))
  tmp=1.0/dist
  c0=-(tMom/(4.0*np.pi))*tmp
  d0=c0*((rx*tmp)**2.0)
  d1=c0*((tmp)-((2.0*rx**2.0)/(dist**3.0)))
  # K(lam) equaion 4.46
  Ka=(a0-rTE0*np.exp(lam0*(Z0-H0)))*a1
  Kb=(b0-rTE1*np.exp(lam1*(Z1-H1)))*lam1

  HxxA=d0*(np.dot(Ka,J0.W[:]))
  HxxB=d1*(np.dot(Kb,J1.W[:]))
    
  Hxx=HxxA+HxxB

  # Free-Space
  Ka=a0*a1
  Kb=b0*lam1

  HxxA=d0*(np.dot(Ka,J0.W[:]))
  HxxB=d1*(np.dot(Kb,J1.W[:]))
    
  Hxx0=HxxA+HxxB
  return Hxx,Hxx0

In [37]:
Hxx,Hxx0=calcHxx(xx, S, z0, rTEj0, rTEj1, u0j0, u0j1,  S.H.J0,  S.H.J1)

In [38]:
H=np.zeros(S.nFreq,dtype=np.complex128) ; H0=np.zeros(S.nFreq,dtype=np.complex128)
if (len(zz) != 0): # Horizontal Co-Planar Coils
  H[zz], H0[zz] = calcHzz(zz, S, z0, rTEj0, u0j0,  S.H.J0)
if (len(xx) != 0): # Vertical Co-Axial Coils
  H[xx], H0[xx] = calcHxx(xx, S, z0, rTEj0, rTEj1, u0j0, u0j1,  S.H.J0,  S.H.J1)
prd=1.e6*((H-H0)/H0)*scl

In [39]:
dH=np.zeros([S.nFreq,nLayers],dtype=np.complex128)
dH0=np.zeros([S.nFreq,nLayers],dtype=np.complex128)

for i in range(nLayers):
  if (len(zz) != 0): # Horizontal Co-Planar Coils
    dH[zz,i], dH0[zz,i] = calcHzz(zz, S, z0, drTEdlogconj0Flg1[:,:,i], u0j0Flg1, S.H.J0)
  if (len(xx) != 0): # Vertical Co-Axial Coils
    dH[xx,i], dH0[xx,i] = calcHxx(xx, S, z0, drTEdlogconj0Flg1[:,:,i], drTEdlogconj1Flg1[:,:,i], u0j0Flg1, u0j1Flg1, S.H.J0, S.H.J1)
scl=tMom*rMom
J=1.e6*np.divide((dH-dH0),dH0)
J*=np.repeat(scl[:,np.newaxis],nLayers,1)

In [40]:
i=30-1
(dH[:,i]-dH0[:,i])/dH0[:,i]

array([  6.96411850e-15 -9.39318463e-15j,
         9.23547380e-28 +1.06824440e-16j,
         4.26357210e-15 -2.99943887e-16j,
        -1.71750459e-25 -4.26871405e-17j,
        -1.54981760e-29 -1.60769010e-22j,  -0.00000000e+00 -0.00000000e+00j])